# 1. Prepare a lot of files

In [1]:
import os

In [2]:
path = 'AA/'
files = ["wiki_0"+str(i) for i in range(9)] + ["wiki_"+str(i) for i in range(10, 14)]

In [3]:
def prep():
    count = 0
    for file in files:
        with open(path+file) as fp: 
            while True:
                markers=0
                empty=True
                with open('done/page'+str(count)+'.txt', 'a') as the_file:
                    while True:
                        line = fp.readline() 

                        if not line:
                            break
                        if line[0]=='<':
                            markers+=1
                            if markers==2:
                                break
                            continue
                        if empty and not line.isspace() and 'http' not in line:
                            empty = False
                        the_file.write(line)
                if empty:
                    os.remove('done/page'+str(count)+'.txt')
                    count-=1
                count+=1
                if not line:
                    break
# prep()

Downloaded a lot of pages from wikipedia xml export.\
Extracted xml to plain text using wiki extractor https://github.com/attardi/wikiextractor .\
Seperated into many files the function prep()

# 2. Prepare Bag-Of-Words
###### In our case union of all words
We will use Lemmatizer

In [4]:
# !pip install num2words
# import nltk
# nltk.download('wordnet')
# nltk.download('stopwords')

In [5]:
import re
import num2words
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [6]:
path = 'done/'
files = ['page'+str(i)+'.txt' for i in range(1388)]
bagOfWords = dict()
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [7]:
def prepare(text):
    text = text.lower()
    text = text.replace("_","")
    text = re.sub(r'[^\w\s]','',text)
    text = re.sub('  +',' ',text)
    text = re.sub(r'½','.5',text) # who tf uses this character?
    text = re.sub(r'¼','.25',text) # who tf uses this character?
    
    text_list = text.split()
    text = ""
    
    for word in text_list:

        word = lemmatizer.lemmatize(word, pos="v")

#         if word.isnumeric():
#             word = num2words.num2words(float(word))
        
        if word in stop_words: continue

        text += word + " "

    text = re.sub(r'[^\w\s]','',text)
    text = re.sub(r'[0-9]','',text)
    
    return text

prepare Function - prepares given text for out bag-of-words, reducing words to their basic dictionary form

In [8]:
count=0
bagOfWords = dict() # slows down really quick, so remember to clear dict
for file in files:
    with open(path+file) as fp:
        for fileline in fp:
            line = prepare(fileline)
            for word in line.split():
                if word not in bagOfWords:
                    bagOfWords[word.lower()]=count
                    count+=1
                    if count % 1e4 == 0:
                        print(count, word)

10000 princesa
20000 obviously
30000 costcutting
40000 sponge
50000 wannabe
60000 gizona


Due to rather slow execution, print to check if everything is in order

# 3. For each document prepare bag-of-words and frequency

In [9]:
count=0
wordBags = [dict() for file in files]
for ix, file in enumerate(files):
    with open(path+file) as fp:
        for fileline in fp:
#             line = re.sub("[^\w]", " ", fileline) 
            line = prepare(fileline)
            for word in line.split():
                if word.lower() not in bagOfWords:
                    continue
                count+=1
                if count % 3e5 == 0:
                    print(count, word)
                if word not in wordBags[ix]:
                    wordBags[ix][word.lower()]=1
                else:
                    wordBags[ix][word.lower()]+=1

300000 coefficients
600000 since
900000 products


Print to assure process not hang

# 4. Create sparse term-by-document matrix

In [10]:
from scipy import sparse

In [11]:
A = sparse.lil_matrix((len(bagOfWords), len(wordBags))) # rows and cols
for ix, bag in enumerate(wordBags):
    for word in bag:
        A[bagOfWords[word], ix] = bag[word]

A - Matrix with the number of bag-of-words rows and number of documents columns

# 5. Calulate Inverse Document Frequency and multiply each matrix element

We want no amplify the strength of rare words / topic related words appearing in specific documents

In [12]:
import math

In [13]:
IDF=0
N=len(files)
nw=1
counted = A.getnnz(axis=1)

In [14]:
count=0
for word in bagOfWords:
    nw=counted[bagOfWords[word]]
    IDF = math.log(N/nw)
#     A[bagOfWords[word]]*=IDF
    A[bagOfWords[word], :]*=IDF
    if count%1e4==0:
        print(word)
    count+=1
#     print(nw, IDF)

japan
subterranean
argument
yasuo
singlecelled
gangsta
manthesingabs


Print to ensure process not hang\
Remember to clear A matrix

# 6. Query
Input words and number of closest documents to display.

We will calculate similarity based on:
q - inputed bag-of-words\
$$ cos\theta_j = \frac{q^Td_j}{||q||*||d_j||} = \frac{q^TAe_j}{||q||*||Ae_j||} $$

In [15]:
query = "Japan and Japanese are super interesting and the best!"
k = 10
# query = "Black people in the Japan"
# k = 20

Input

In [16]:
import numpy as np

In [17]:
querybag = np.array([0 for i in bagOfWords])
text = prepare(query).split()


for word in text:
    if word not in bagOfWords:
        text.remove(word)
    querybag[bagOfWords[word]]+=1
print(len(bagOfWords), len(wordBags))

66262 1388


Input preperation

In [18]:
res = []
mod = np.linalg.norm(querybag)

colA = A.tocsc()
for i in range(len(wordBags)): # number of columns/files 
    modj = sparse.linalg.norm(colA[:,i])
    
    res.append(((querybag.T@colA[:,i])/(mod*modj) if modj!=0 else 0, i))
        
    if i%1e2==0:
        print(i)

res.sort(key=lambda x: (-x[0], -x[1]))
# res.sort()
res = res[:k]
print(res)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
[(array([0.50857295]), 0), (array([0.41311798]), 2), (array([0.25316546]), 8), (array([0.24275078]), 6), (array([0.19001917]), 1065), (array([0.14164975]), 3), (array([0.1381964]), 337), (array([0.11915394]), 338), (array([0.11407905]), 112), (array([0.10533123]), 354)]


Calculated results

In [19]:
print(f"For query \"{query}\", with key words {text}, the best {k} results are:\n")
for ix, i in enumerate(res):
    print(f"{ix+1}: file page{i[1]}.txt titled:")
    with open("done/page"+str(i[1])+".txt", "r") as f:
        print(f.readline())

For query "Japan and Japanese are super interesting and the best!", with key words ['japan', 'japanese', 'super', 'interest', 'best'], the best 10 results are:

1: file page0.txt titled:
Japan

2: file page2.txt titled:
Outline of Japan

3: file page8.txt titled:
Japan and World Bank

4: file page6.txt titled:
Black people in Japan

5: file page1065.txt titled:
Interest articulation

6: file page3.txt titled:
Blakiston's Line

7: file page337.txt titled:
History of anime

8: file page338.txt titled:
Anime

9: file page112.txt titled:
Tibet Interest Group

10: file page354.txt titled:
Japan Media Arts Festival



# 7. Normalize vectors $d_j$ and $q$

Then aforementioned equation is simplified to
$$ |q^TA| = [|cos\theta_1|,|cos\theta_2|,\dots,|cos\theta_n|] $$

In [20]:
query = "Japan and Japanese are super interesting and the best!"
k = 10

Input

In [21]:
colA = A.tocsc()
querybag = np.array([0 for i in bagOfWords])
text = prepare(query).split()


for word in text:
    if word not in bagOfWords:
        text.remove(word)
    querybag[bagOfWords[word]]+=1
print(len(bagOfWords), len(wordBags))

66262 1388


Input processing

In [22]:
querybag = querybag/np.linalg.norm(querybag) #remember to clean!

for i in range(len(wordBags)):
    norm = sparse.linalg.norm(colA[:,i])
    colA[:,i] = colA[:,i]/norm
    if i%1e2 == 0:
        print(i, norm)

res = list(querybag.T@colA)
for i in range(len(res)):
    res[i]=(res[i], i)
res.sort(key=lambda x: (-x[0], -x[1]))
# res.sort()
res = res[:k]
print(res)

0 834.4732328418058
100 276.64221464214324
200 119.45020182290006
300 61.74315362409992
400 68.05379464717565
500 105.94348595433729
600 39.84752023403267
700 471.6939809301491
800 30.434632017645008
900 28.24674182487699
1000 70.67316282378326
1100 122.6489685848564
1200 273.0646039960876
1300 202.8068188279269
[(0.5085729483167328, 0), (0.4131179764654834, 2), (0.2531654597053116, 8), (0.24275077689544935, 6), (0.19001917387558614, 1065), (0.14164974838755934, 3), (0.1381963986947751, 337), (0.11915394089685416, 338), (0.11407904859180991, 112), (0.10533122744361345, 354)]


In [23]:
print(f"For query {query}, with key words {text}, the best {k} results are:")
for ix, i in enumerate(res):
    print(f"{ix+1}: file page{i[1]}.txt titled:")
    with open("done/page"+str(i[1])+".txt", "r") as f:
        print(f.readline())

For query Japan and Japanese are super interesting and the best!, with key words ['japan', 'japanese', 'super', 'interest', 'best'], the best 10 results are:
1: file page0.txt titled:
Japan

2: file page2.txt titled:
Outline of Japan

3: file page8.txt titled:
Japan and World Bank

4: file page6.txt titled:
Black people in Japan

5: file page1065.txt titled:
Interest articulation

6: file page3.txt titled:
Blakiston's Line

7: file page337.txt titled:
History of anime

8: file page338.txt titled:
Anime

9: file page112.txt titled:
Tibet Interest Group

10: file page354.txt titled:
Japan Media Arts Festival



# 8. Noise reduction using SVD and low rank approximation

$$ A\simeq A_k = U_kD_kV^T_k = \sum_{i=1}^{k}\sigma_iu_iv_i^T $$

And probability measure being

$$ cos\phi_j = \frac{q^TA_ke_j}{||q||*||A_ke_j||} $$

# 9. Compare reuslts with and without noise reduction. Find optimal k and test IDF's impact on results

To test IDF's impact on results I omitted the cells in the 5th point.\
It resulted in less in less accurate outcome.\
IDF has huge impact on search resuts, it devalues popular words, making other (rarer, unique) have bigger influence on the result.